In [1]:
import numpy as np
import pandas as pd
import gensim as gs
from gensim.models import Word2Vec as wv

from operator import itemgetter

In [2]:
without_time = True

if without_time:
    caseFilename = "../cleaned_data/case.csv"
else:
    caseFilename = "../cleaned_data/case_w_time.csv"

In [3]:
caseGD = pd.read_csv(caseFilename, sep=',')
caseGD.head()

,Unnamed: 0,subject_id,time,eve_index
0,0,10026821,2134-06-02 07:12:00,26406
1,1,10026821,2134-06-02 07:12:00,6699
2,2,10026821,2134-06-02 07:12:00,5645
3,3,10026821,2134-06-02 07:12:00,5612
4,4,10026821,2134-06-03 08:00:00,35454


In [4]:
caseGD = caseGD.groupby(['subject_id', 'eve_index']).size().reset_index()
caseGD["GD"] = 1
caseGD.columns = (['subject_id', 'eve_index','value','GD'])
caseGD.head()

,subject_id,eve_index,value,GD
0,10026821,5612,1,1
1,10026821,5645,1,1
2,10026821,6699,1,1
3,10026821,26406,1,1
4,10026821,28623,1,1


In [5]:
if without_time:
    controlFilename = "../cleaned_data/control.csv"
else:
    controlFilename = "../cleaned_data/control_w_time.csv"

In [6]:
controlGD = pd.read_csv(controlFilename, sep=',')
controlGD.head()

,Unnamed: 0,subject_id,time,eve_index
0,0,10006196,2204-09-22 18:42:00,5353
1,1,10006196,2204-09-22 19:00:00,28623
2,2,10006196,2204-09-22 19:00:00,30848
3,3,10006196,2204-09-22 19:00:00,29794
4,4,10006196,2204-09-22 19:00:00,33424


In [7]:
controlGD = controlGD.groupby(['subject_id', 'eve_index']).size().reset_index()
controlGD["GD"] = 0
controlGD.columns = (['subject_id', 'eve_index','value','GD'])
controlGD.head()

,subject_id,eve_index,value,GD
0,10006196,5353,1,0
1,10006196,5362,1,0
2,10006196,5496,1,0
3,10006196,5683,1,0
4,10006196,5702,1,0


In [8]:
ccGD = pd.concat([caseGD, controlGD], ignore_index=True)

In [9]:
ccGD.head()

,subject_id,eve_index,value,GD
0,10026821,5612,1,1
1,10026821,5645,1,1
2,10026821,6699,1,1
3,10026821,26406,1,1
4,10026821,28623,1,1


In [10]:
print(ccGD.eve_index.values.max())

36765


In [11]:
w2vFilename = "../word2vec_model/w2vmodel" # number of "words" 36802
# w2vFilename = "word2vec_model/w2vmodel_mv" # number of "words" 10932
#w2vFilename = "../word2vec_model/w2vmodel_mv_wt" # number of "words" 28121
#w2vFilename = "../word2vec_model/w2vmodel_wt" # number of "words" 29391

In [12]:
model = wv.load(w2vFilename)

In [13]:
numEvents = 36801

In [14]:
model.wv['67']

array([ 0.01709885, -0.01908228,  0.7339222 ,  0.08540019, -0.5077995 ,
       -0.1203943 , -0.13808256,  0.49743292,  0.16887338, -0.01942202,
        0.3151415 ,  0.13579528, -0.22905347,  0.02272777, -0.13646948,
        0.7201578 , -0.45984256, -0.44264024,  0.16334397,  0.40282223,
       -0.8054902 , -0.5090025 , -0.966486  ,  0.15755685, -0.34735972,
        0.2193293 ,  0.3013564 , -0.73969626,  0.09845282, -0.05367937,
        0.5437    ,  0.19661953, -0.12846404,  1.0680841 , -0.58144987,
        1.0527238 ,  0.07684191,  0.6233936 , -0.31454572,  0.45793232,
        0.8358476 , -0.84131986,  0.35347652,  0.06646153,  0.3203047 ,
       -0.02213138, -0.1813543 ,  0.04670492, -0.6084616 , -1.1480337 ,
        0.42845124, -0.28957456, -0.41049716, -0.206617  , -0.9400068 ,
        0.72087497, -0.2340225 ,  0.90789044,  0.29173025, -0.6658276 ,
        0.13487521, -0.3353092 ,  0.11201029,  0.13519748,  0.35634774,
       -0.23755546, -0.3775757 ,  0.56638616,  0.53067   , -0.54

In [15]:
w2vList = []
for event in range(1,numEvents+1):
    w2vList.append([event,np.mean(model.wv[str(event)])])

In [16]:
w2vDF = pd.DataFrame(w2vList)
w2vDF.columns = (['eve_index','value'])
w2vDF.head()

,eve_index,value
0,1,0.005268
1,2,-0.016853
2,3,-0.013744
3,4,-0.013886
4,5,-0.003011


In [19]:
print(fullGD.eve_index.values.max())

36765


In [17]:
fullGD = pd.merge(ccGD, w2vDF, on='eve_index')
fullGD.head()

,subject_id,eve_index,value_x,GD,value_y
0,10026821,5612,1,1,-0.045744
1,15247114,5612,1,1,-0.045744
2,19075055,5612,1,1,-0.045744
3,10031873,5612,1,0,-0.045744
4,15101291,5612,1,0,-0.045744


In [18]:
fullGD = fullGD[['subject_id','eve_index','value_y','GD']]
fullGD.columns = (['subject_id','eve_index','value','GD'])
fullGD.head()

,subject_id,eve_index,value,GD
0,10026821,5612,-0.045744,1
1,15247114,5612,-0.045744,1
2,19075055,5612,-0.045744,1
3,10031873,5612,-0.045744,0
4,15101291,5612,-0.045744,0


In [29]:
patient_features = {}

for row in fullGD.itertuples():
    if row[1] not in patient_features:
        patient_features[row[1]] = []
    entry = (row[2], row[3])
    patient_features[row[1]].append(entry)

hfGD = fullGD[['subject_id', 'GD']]

hfLabel = {}
for row in hfGD.itertuples():
    if row[1] not in hfLabel:
        if row[2] == 1:
            hfLabel[row[1]] = 1
        else:
            hfLabel[row[1]] = 0

In [30]:
hfLabel

{10026821: 1,
 15247114: 1,
 19075055: 1,
 10031873: 0,
 15101291: 0,
 10421857: 1,
 10678740: 1,
 11930434: 1,
 11942140: 1,
 12733064: 1,
 13133103: 1,
 13381472: 1,
 13414137: 1,
 13476077: 1,
 13578437: 1,
 17206728: 1,
 11925886: 0,
 16048023: 0,
 17272916: 0,
 17312302: 0,
 17372646: 0,
 17513330: 0,
 18263620: 0,
 10078279: 1,
 10438339: 1,
 11625331: 1,
 11754464: 1,
 12161040: 1,
 12263134: 1,
 12512858: 1,
 12575245: 1,
 12956635: 1,
 13043552: 1,
 13203222: 1,
 13219280: 1,
 13458451: 1,
 13558573: 1,
 13941452: 1,
 14318477: 1,
 14368020: 1,
 14507638: 1,
 16408366: 1,
 16880902: 1,
 17497258: 1,
 19008436: 1,
 10056202: 0,
 10302166: 0,
 10375101: 0,
 11224954: 0,
 12382932: 0,
 13003077: 0,
 13025966: 0,
 13055747: 0,
 14052090: 0,
 14382671: 0,
 14472118: 0,
 14762915: 0,
 15903988: 0,
 16814772: 0,
 17650229: 0,
 17975369: 0,
 18209473: 0,
 10494796: 1,
 10578144: 1,
 10751849: 1,
 10882364: 1,
 10949638: 1,
 11168175: 1,
 11953378: 1,
 11984314: 1,
 12682028: 1,
 12987

In [31]:
patient_features

{10026821: [(5612, -0.04574382305145264),
  (5645, -0.045210178941488266),
  (6699, -0.031315721571445465),
  (26406, -0.03075406327843666),
  (28623, 0.004221947863698006),
  (29535, -0.0348292738199234),
  (29794, -0.014380463398993015),
  (30848, -0.0017574111698195338),
  (32075, 0.0033208783715963364),
  (32554, -0.005958068184554577),
  (32609, 0.0002714693546295166),
  (33355, -0.048518262803554535),
  (33424, -0.012906261719763279),
  (33649, -0.025417886674404144),
  (34157, -0.03293038159608841),
  (34261, 0.01767483539879322),
  (35388, 0.0019516993779689074),
  (35454, -0.0024365473072975874),
  (35881, -0.030688203871250153)],
 15247114: [(5612, -0.04574382305145264),
  (26406, -0.03075406327843666),
  (28623, 0.004221947863698006),
  (29535, -0.0348292738199234),
  (29794, -0.014380463398993015),
  (30848, -0.0017574111698195338),
  (32075, 0.0033208783715963364),
  (32609, 0.0002714693546295166),
  (33424, -0.012906261719763279),
  (34261, 0.01767483539879322),
  (35388,

In [32]:
if without_time:
    filename1 = "../cleaned_data/features_svmlight_w2v.train"
    filename2 = "../cleaned_data/features_w2v.train"
else:
    filename1 = "../cleaned_data/features_svmlight_w2v_wt.train"
    filename2 = "../cleaned_data/features_w2v_wt.train"

In [33]:
fileWriter1 = open(filename1, 'wb')
fileWriter2 = open(filename2, 'wb')

for patient in sorted(patient_features):
    fileWriter1.write('{:.0f}'.format(hfLabel[patient]).encode())
    fileWriter2.write('{:.0f} {:.0f}'.format(patient, hfLabel[patient]).encode())
    for record in sorted(patient_features[patient], key=itemgetter(0)):
        fileWriter1.write(' {:.0f}:{:.6f}'.format(record[0], record[1]).encode())
        fileWriter2.write(' {:.0f}:{:.6f}'.format(record[0], record[1]).encode())
    fileWriter1.write(" \n".encode())
    fileWriter2.write(" \n".encode())

fileWriter2.close()
fileWriter1.close()